# Document Processing

Let's see how to connect documents to our graph using the Knowledge Graph Builder.

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

load_dotenv()
driver = GraphDatabase.driver(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
)

## Connect Product Manual

In [ ]:
def add_product_manual(product_id, file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    
    with driver.session() as session:
        result = session.run("""
        MATCH (p:Product {product_id: $product_id})
        CREATE (d:Document {
            id: randomUUID(),
            type: 'manual',
            content: $content,
            created_at: datetime()
        })
        CREATE (p)-[:HAS_MANUAL]->(d)
        RETURN d.id as doc_id
        """, product_id=product_id, content=content)
        
        return result.single()['doc_id']

# Add Laptop Pro manual
manual_id = add_product_manual('P1', '../data/documents/laptop_pro_manual.txt')
print(f"Added manual with ID: {manual_id}")

## Connect Support Case

In [ ]:
def add_support_case(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    
    with driver.session() as session:
        result = session.run("""
        CREATE (d:Document {
            id: randomUUID(),
            type: 'support_case',
            content: $content,
            created_at: datetime()
        })
        WITH d
        MATCH (c:Customer {customer_id: 'C1'})
        MATCH (p:Product {product_id: 'P1'})
        CREATE (c)-[:HAS_CASE]->(d)
        CREATE (d)-[:ABOUT]->(p)
        RETURN d.id as doc_id
        """)
        
        return result.single()['doc_id']

# Add support case
case_id = add_support_case('../data/documents/support_case_1.txt')
print(f"Added support case with ID: {case_id}")

## Query Connected Documents

In [ ]:
def get_product_documents(product_id):
    with driver.session() as session:
        result = session.run("""
        MATCH (p:Product {product_id: $product_id})
        MATCH (p)-[:HAS_MANUAL|ABOUT]-(d:Document)
        RETURN d.type as doc_type,
               d.content as content
        """, product_id=product_id)
        
        return list(result)

# Get documents for Laptop Pro
docs = get_product_documents('P1')
for doc in docs:
    print(f"\nDocument Type: {doc['doc_type']}")
    print(f"Content Preview: {doc['content'][:100]}...")

## Next Steps

Now that we have documents connected to our graph, let's explore GraphRAG patterns!